In [1]:
import pandas as pd
import numpy as np

In [2]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df = pd.read_pickle(arquivo)
df.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df.rename(columns={'sexo': 'SEXO'}, inplace=True)

In [5]:
df[['SEXO', 'L0']].groupby(by=['SEXO']).count().rename({'L0': 'QTDE'}, axis=1)

,QTDE
SEXO,
0,4376
1,4335
9,1015


In [7]:
X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (9726, 16) (9726,)


In [8]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (6516, 16) (3210, 16) (6516,) (3210,)


In [54]:
# treinar com todos os dados disponíveis

X_train, y_train = X, y

print("Shapes:", X_train.shape, y_train.shape)

Shapes: (9726, 4) (9726,)


https://automl.github.io/auto-sklearn/master/manual.html

In [73]:
# https://automl.github.io/auto-sklearn/master/installation.html

#!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
#!pip3 install auto-sklearn

In [9]:
import autosklearn.classification

/home/rodrigo/anaconda3/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [10]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=20*60,
    per_run_time_limit=2*60,
    n_jobs=2,
    memory_limit=1024,
    seed=42,
    include_preprocessors=["no_preprocessing"]
)
automl.fit(X_train, y_train)

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=1024, n_jobs=2, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [11]:
print(automl.show_models())

[(0.380000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'minmax', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:adaboost:algorithm': 'SAMME', 'classifier:adaboost:learning_rate': 0.02607458507935197, 'classifier:adaboost:max_depth': 8, 'classifier:adaboost:n_estimators': 97, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.0012475734118542873},
dataset_properties={
  'task': 2,
  'sparse': False,
  'multilabel': False,
  'multiclass': True,
  'target_type': 'classification',
  'signed': False})),
(0.280000, SimpleClassificationPi

In [12]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: ce3440f6-9c7f-11eb-8c2b-6fa7d7024e68
  Metric: accuracy
  Best validation score: 0.950721
  Number of target algorithm runs: 73
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0



In [13]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print("Best configuration:", automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]])

Lowest loss: 0.04927940492794047
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'gradient_boosting'
  classifier:gradient_boosting:early_stop, Value: 'off'
  classifier:gradient_boosting:l2_regularization, Value: 2.8901356598954326e-09
  classifier:gradient_boosting:learning_rate, Value: 0.04148598290965726
  classifier:gradient_boosting:loss, Constant: 'auto'
  classifier:gradient_boosting:max_bins, Constant: 255
  classifier:gradient_boosting:max_depth, Constant: 'None'
  classifier:gradient_boosting:max_leaf_nodes, Value: 58
  classifier:gradient_boosting:min_samples_leaf, Value: 4
  classifier:gradient_boosting:scoring, Constant: 'loss'
  classifier:gradient_boosting:tol, Constant: 1e-07
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'no_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'no_coalescense'
  data_preprocessing:numerical_transformer

In [20]:
# treinar o melhor modelo com todos os dados
automl.fit(X, y)

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=1024, n_jobs=2, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=1200)

In [58]:
# carregar dados de teste

In [21]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [22]:
df2.shape

(48970, 17)

In [23]:
X_test = df2.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [24]:
y_pred = automl.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 1, 0], dtype=uint8)

In [25]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 96.71 %


In [26]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[21543   116   232]
 [   52 19024   183]
 [  471   558  6791]]
